<hr>
<hr>
<div class="jumbotron jumbotron-fluid">
  <div class="container">
    <h1 class="display-4">Проект по SQL</h1>
  </div>
</div>

## <a name="content">Содержание</a>
- [0. Знакомство с проектом](#0)
- [1. Исследуем таблицы](#1)
- [2. Запросы по задаче](#2)
 - [2.1. Посчитаем, сколько книг вышло после 1 января 2000 года](#2.1)
 - [2.2. Для каждой книги посчитаем количество обзоров и среднюю оценку](#2.2)
 - [2.3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры](#2.3)
 - [2.4. Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками](#2.4)
 - [2.5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#2.5)

<hr>
<hr>

## 0. Знакомство с проектом  <a name="0"></a>  
<a class="btn btn-primary btn-sm" href="#content" role="button">Вернуться к содержанию</a>
____

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных
_____
**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### ER-диаграмма

![Описание картинки](https://downloader.disk.yandex.ru/preview/d2e478d706397d5e7bd946d15ee8759a24a8d6d850c9e20b79fc8f7ee650cd2d/5fa7ca81/TfskC9wHP_msUMASvZvFsLZtaykGIRE37UlwUD4Ryk3Z3nP5J57MdKf3jUhg9eG-0obsp_NzLITG-hqyBL4G6Q==?uid=0&filename=final_project_SQL_ER.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&tknv=v2&owner_uid=73458723&size=2048x2048)

### Рассчет
_____
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### План
______
- Опишем цели исследования;
- Исследуем таблицы — выведем первые строки;
- Сделаем по одному SQL-запросу для решения каждого задания;
- Выведем результат каждого запроса в тетрадке;
- Опишем выводы по каждой из решённых задач.

## 1. Исследуем таблицы  <a name="1"></a>  
<a class="btn btn-primary btn-sm" href="#content" role="button">Вернуться к содержанию</a>
____

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           5 non-null      int64 
 1   author_id         5 non-null      int64 
 2   title             5 non-null      object
 3   num_pages         5 non-null      int64 
 4   publication_date  5 non-null      object
 5   publisher_id      5 non-null      int64 
dtypes: int64(4), object(2)
memory usage: 368.0+ bytes


In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  5 non-null      int64 
 1   author     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [9]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  5 non-null      int64 
 1   publisher     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [7]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   rating     5 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 288.0+ bytes


In [8]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   text       5 non-null      object
dtypes: int64(2), object(2)
memory usage: 288.0+ bytes


## 2. Запросы по задаче<a name="2"></a>  
<a class="btn btn-primary btn-sm" href="#content" role="button">Вернуться к содержанию</a>
____
### 2.1. Посчитаем, сколько книг вышло после 1 января 2000 года<a name="2.1"></a>

In [7]:
query = '''
SELECT
    COUNT(DISTINCT book_id) AS book_cnt
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,book_cnt
0,819


После 1 января 2000 года (не включительно) вышла 819 книг.

### 2.2. Для каждой книги посчитаем количество обзоров и среднюю оценку<a name="2.2"></a>
<a class="btn btn-primary btn-sm" href="#content" role="button">Вернуться к содержанию</a>

In [8]:
query = '''
SELECT
    books.book_id AS id,
    books.title AS title,
    COUNT(DISTINCT reviews.review_id) AS cnt_reviews,
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM
    books
INNER JOIN reviews ON reviews.book_id = books.book_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY id
ORDER BY  avg_rating DESC, cnt_reviews DESC
'''
pd.io.sql.read_sql(query, con = engine)

,id,title,cnt_reviews,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,182,Dead Souls,2,5.00
...,...,...,...,...
989,915,The World Is Flat: A Brief History of the Twen...,3,2.25
990,202,Drowning Ruth,3,2.00
991,316,His Excellency: George Washington,2,2.00
992,371,Junky,2,2.00


### 2.3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры<a name="2.3"></a>
<a class="btn btn-primary btn-sm" href="#content" role="button">Вернуться к содержанию</a>

In [9]:
query = '''
SELECT
    publishers.publisher as publisher,
    COUNT(books.publisher_id) as cnt
FROM
    books
INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE
    books.num_pages > 50
GROUP BY publisher
ORDER BY cnt DESC
LIMIT 10
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


Penguin Books выпустило больше всего книг толще 50 страниц.

### 2.4. Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками<a name="2.4"></a>
<a class="btn btn-primary btn-sm" href="#content" role="button">Вернуться к содержанию</a>

In [10]:
query = '''
SELECT 
    authors.author,
    ROUND(AVG(rating), 2) AS avg_rating,
    COUNT(rating_id) AS cnt_rating
FROM
    authors
INNER JOIN books ON authors.author_id = books.author_id
AND books.book_id IN (
                    SELECT
                        book_id
                    FROM
                        ratings
                    GROUP BY
                        book_id
                    HAVING COUNT(rating_id) >= 50
                    )
INNER JOIN ratings ON books.book_id = ratings.book_id
GROUP BY
    authors.author
ORDER BY AVG(rating) DESC
    
'''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating,cnt_rating
0,J.K. Rowling/Mary GrandPré,4.29,310
1,Markus Zusak/Cao Xuân Việt Khương,4.26,53
2,J.R.R. Tolkien,4.25,162
3,Louisa May Alcott,4.19,52
4,Rick Riordan,4.08,62
5,William Golding,3.90,71
6,J.D. Salinger,3.83,86
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79,57
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.79,66
9,Lois Lowry,3.75,56


Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг. 

### 2.5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок<a name="2.5"></a>
<a class="btn btn-primary btn-sm" href="#content" role="button">Вернуться к содержанию</a>

In [11]:
query = '''
SELECT
    'Cреднее количество обзоров от пользователей:',
    ROUND(AVG(cnt_reviews), 2) AS avg_cnt_reviews
FROM
    (SELECT
        (SELECT
            COUNT(*)
        FROM
            reviews
        WHERE
            username = ratings.username
        ) AS cnt_reviews
    FROM
        ratings
    GROUP BY
        username
    HAVING
    COUNT(*) >= 50
    ) AS SUBQ
'''
pd.io.sql.read_sql(query, con = engine)

,?column?,avg_cnt_reviews
0,Cреднее количество обзоров от пользователей:,24.22
